- link prediction 

In [1]:
import pickle

pk = '/home/jina/reprod/data/pickle/train_ER2'
# pk = '/home/jina/reprod/data/pickle/train_ER'
# pk = '/home/jina/reprod/data/pickle/dense_test'
# pk = '/home/jina/reprod/data/pickle/train_1'
# pk = '/home/jina/reprod/data/pickle/new_1'
with open(pk, 'rb') as f:
    train_data = pickle.load(f)

# valid_data = train_data
# pk_v = '/home/jina/reprod/data/pickle/valid_ER'
pk_v = '/home/jina/reprod/data/pickle/valid_1'
# pk_v = '/home/jina/reprod/data/pickle/new_1'
with open(pk_v, 'rb') as f:
    valid_data = pickle.load(f)

In [32]:
train_data

[Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[100, 11], edge_index=[2, 4950], edge_attr=[4950, 100], y=[4950], num_nodes=100),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[50, 11], edge_index=[2, 1225], edge_attr=[1225, 100], y=[1225], num_nodes=50),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 D

In [33]:
valid_data

[Data(x=[20, 11], edge_index=[2, 190], edge_attr=[190, 100], y=[190], num_nodes=20),
 Data(x=[20, 11], edge_index=[2, 190], edge_attr=[190, 100], y=[190], num_nodes=20),
 Data(x=[50, 11], edge_index=[2, 1225], edge_attr=[1225, 100], y=[1225], num_nodes=50),
 Data(x=[100, 11], edge_index=[2, 4950], edge_attr=[4950, 100], y=[4950], num_nodes=100),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[100, 11], edge_index=[2, 4950], edge_attr=[4950, 100], y=[4950], num_nodes=100),
 Data(x=[20, 11], edge_index=[2, 190], edge_attr=[190, 100], y=[190], num_nodes=20),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10),
 Data(x=[50, 11], edge_index=[2, 1225], edge_attr=[1225, 100], y=[1225], num_nodes=50),
 Data(x=[10, 11], edge_index=[2, 45], edge_attr=[45, 100], y=[45], num_nodes=10)]

In [34]:
from torch_geometric.nn import SAGEConv, BatchNorm
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np 
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, residual=True):
        super(GraphSAGE, self).__init__()
        self.num_layers = num_layers
        self.residual = residual
        
        # Create convolutional layers dynamically based on num_layers
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        
        # First layer
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns.append(BatchNorm(hidden_channels))
        
        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.bns.append(BatchNorm(hidden_channels))
        
        # Last layer
        self.convs.append(SAGEConv(hidden_channels, out_channels))
        
        # Linear layer for dimension matching if needed
        if residual and in_channels != out_channels:
            self.dim_match = nn.Linear(in_channels, out_channels)
        else:
            self.dim_match = None

    def encode(self, x, edge_index):
        x_res = x  # For potential residual connection
        
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=0.3, training=self.training)
        
        # Apply the last convolutional layer without activation for output
        x = self.convs[-1](x, edge_index)
        
        # Residual connection after the last hidden layer if enabled
        if self.residual and self.num_layers > 2:
            if self.dim_match is not None:
                x_res = self.dim_match(x_res)  # Adjust dimension if necessary
            x += x_res

        return x
    
    def decode(self, z, edge_index):
        z_product = torch.sum(z[edge_index[0]] * z[edge_index[1]], dim=1)
        return z_product / (torch.norm(z[edge_index[0]], dim=1) * torch.norm(z[edge_index[1]], dim=1) + 1e-6)  

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        return z


In [35]:

# Random seed setting
def seed_everything(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

seed_everything(47)

# Model, optimizer, loss initialization
in_channels = train_data[0].num_features
hidden_channels = 64  # Increased hidden channel size
out_channels = 32
model = GraphSAGE(in_channels, hidden_channels, out_channels, num_layers=3)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)  # Adjusted learning rate for stability
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
criterion = nn.BCEWithLogitsLoss()  # Using BCEWithLogitsLoss since sigmoid is not applied in decode

def train(train_data):
    model.train()
    total_loss = 0
    total_samples = 0
    all_scores = []
    all_labels = []
    for data in train_data:
        optimizer.zero_grad()
        edge_index = data.edge_index
        z = model(data.x.float(), edge_index)
        # print(z.shape)
        data.y[data.y==2] = 1
        
        # Separate positive and negative edges based on labels
        # pos_edge_index = edge_index[:, data.y == 1]
        # neg_edge_index = edge_index[:, data.y == 0]
        
        # Ensure equal number of negative samples as positive samples
        # num_pos = pos_edge_index.size(1)
        # if neg_edge_index.size(1) > num_pos * 2:  # Match negative samples to positive samples
        #     perm = torch.randperm(neg_edge_index.size(1))[:num_pos * 2]  # Random permutation for sampling
        #     neg_edge_index = neg_edge_index[:, perm]
        
        # pos_score = model.decode(z, pos_edge_index)
        # neg_score = model.decode(z, neg_edge_index)
        # print(pos_score.shape, neg_score.shape)
        # print(neg_score)
        
        score = model.decode(z, edge_index)

        # pos_labels = torch.ones(pos_score.size(0), device=pos_score.device).float()
        # neg_labels = torch.zeros(neg_score.size(0), device=neg_score.device).float()
        
        # all_scores.append(pos_score)
        # all_scores.append(neg_score)
        # all_labels.append(pos_labels)
        # all_labels.append(neg_labels)
        all_scores.append(score)
        all_labels.append(data.y)

        # all_score = torch.cat([pos_score, neg_score], dim=0)
        # all_label = torch.cat([pos_labels, neg_labels], dim=0)

        loss = criterion(score, data.y)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        # total_loss += loss.item() * (pos_labels.size(0) + neg_labels.size(0))
        total_loss += loss.item() * (data.y.size(0))
        # total_samples += (pos_labels.size(0) + neg_labels.size(0))
        total_samples += data.y.size(0)
        
    all_scores = torch.cat(tuple(all_scores), dim=0).detach().cpu().numpy()
    all_labels = torch.cat(tuple(all_labels), dim=0).detach().cpu().numpy()
    predictions = (torch.sigmoid(torch.tensor(all_scores)) > 0.5).int().numpy()

    accuracy = accuracy_score(all_labels, predictions)
    f1 = f1_score(all_labels, predictions)

    return total_loss / total_samples, accuracy, f1

# Testing loop
def test(valid_data):
    model.eval()
    all_scores = []
    all_labels = []
    with torch.no_grad():
        for data in valid_data:
            edge_index = data.edge_index
            data.y[data.y==2] = 1
            z = model(data.x.float(), edge_index)

            out = model.decode(z, edge_index).view(-1).sigmoid()
            
            # Append results to all_scores and all_labels
            all_scores.extend(out.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())  # Assuming data.y is the label tensor

    # Ensure labels are binary
    if len(np.unique(all_labels)) > 2:
        raise ValueError("Expected binary labels but found multi-class labels. Check your data.")
        
    # Calculate metrics
    auc = roc_auc_score(all_labels, all_scores, average="macro")
    predictions = [1 if score > 0.7 else 0 for score in all_scores]
    accuracy = accuracy_score(all_labels, predictions)
    f1 = f1_score(all_labels, predictions)

    return auc, accuracy, f1, predictions, all_labels

# Run training for all epochs
epochs = 200
for epoch in range(epochs):
    train_loss, train_accuracy, train_f1 = train(train_data)
    scheduler.step(train_loss)  # Update learning rate based on training loss
    if epoch % 10 == 0:
        auc, accuracy, f1, predicted, all_labels = test(valid_data)
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Test AUC: {auc:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')


# Final Test
auc, accuracy, f1, predicted, all_labels = test(valid_data)
print(f'Final Test AUC: {auc:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')


/home/jina/anaconda3/envs/jina_gnn/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Loss: 0.7106, Train Accuracy: 0.5702, Train F1 Score: 0.7165, Test AUC: 0.5417, Test Accuracy: 0.6868, Test F1 Score: 0.0956
Epoch 10, Loss: 0.6968, Train Accuracy: 0.5696, Train F1 Score: 0.7032, Test AUC: 0.5164, Test Accuracy: 0.6967, Test F1 Score: 0.0846
Epoch 20, Loss: 0.6945, Train Accuracy: 0.5718, Train F1 Score: 0.7028, Test AUC: 0.5106, Test Accuracy: 0.6931, Test F1 Score: 0.0994
Epoch 30, Loss: 0.6946, Train Accuracy: 0.5769, Train F1 Score: 0.7131, Test AUC: 0.5227, Test Accuracy: 0.7088, Test F1 Score: 0.0869
Epoch 40, Loss: 0.6926, Train Accuracy: 0.5743, Train F1 Score: 0.7023, Test AUC: 0.5211, Test Accuracy: 0.7132, Test F1 Score: 0.0996
Epoch 50, Loss: 0.6882, Train Accuracy: 0.5769, Train F1 Score: 0.6998, Test AUC: 0.4996, Test Accuracy: 0.7151, Test F1 Score: 0.0873
Epoch 60, Loss: 0.6888, Train Accuracy: 0.5762, Train F1 Score: 0.6985, Test AUC: 0.4795, Test Accuracy: 0.6971, Test F1 Score: 0.0779
Epoch 70, Loss: 0.6862, Train Accuracy: 0.5765, Train F1

In [36]:
for a, b in zip(predicted, all_labels):
    print(a, b)

1 0.0
1 0.0
0 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
0 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
0 0.0
1 0.0
0 1.0
0 0.0
1 0.0
0 1.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.

In [37]:
# Convert to numpy arrays for counting values and cast to integer type
predicted_flat = np.array(predicted).astype(int)
all_labels_flat = np.array(all_labels).astype(int)

# Count values
count_predicted = np.bincount(predicted_flat)
count_labels = np.bincount(all_labels_flat)

# Print counts
print("Predicted counts:", count_predicted)
print("Actual labels counts:", count_labels)

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(all_labels_flat, predicted_flat)
precision = precision_score(all_labels_flat, predicted_flat)
recall = recall_score(all_labels_flat, predicted_flat)
f1 = f1_score(all_labels_flat, predicted_flat)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

tn, fp, fn, tp = confusion_matrix(all_labels_flat, predicted_flat).ravel()
print(tn, fp, fn, tp)

Predicted counts: [10000  3055]
Actual labels counts: [12110   945]
Accuracy: 0.7132
Precision: 0.0419
Recall: 0.1354
F1 Score: 0.0640
9183 2927 817 128


In [22]:

# Random seed setting
def seed_everything(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

seed_everything(47)

# Model, optimizer, loss initialization
in_channels = train_data[0].num_features
hidden_channels = 16  # Increased hidden channel size
out_channels = 4
model = GraphSAGE(in_channels, hidden_channels, out_channels, num_layers=3)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)  # Adjusted learning rate for stability
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
criterion = nn.BCEWithLogitsLoss()  # Using BCEWithLogitsLoss since sigmoid is not applied in decode

def train(train_data):
    model.train()
    total_loss = 0
    total_samples = 0
    all_scores = []
    all_labels = []
    for data in train_data:
        optimizer.zero_grad()
        edge_index = data.edge_index
        # z = model(data.x.float(), edge_index)
        data.y[data.y==2] = 1
        
        # Separate positive and negative edges based on labels
        pos_edge_index = edge_index[:, data.y == 1]
        neg_edge_index = edge_index[:, data.y == 0]
        
        # Ensure equal number of negative samples as positive samples
        # num_pos = pos_edge_index.size(1)
        # if neg_edge_index.size(1) > num_pos * 2:  # Match negative samples to positive samples
        #     perm = torch.randperm(neg_edge_index.size(1))[:num_pos * 2]  # Random permutation for sampling
        #     neg_edge_index = neg_edge_index[:, perm]
        
        z_p = model(data.x.float(), pos_edge_index)
        z_n = model(data.x.float(), neg_edge_index)

        pos_score = model.decode(z_p, pos_edge_index)
        neg_score = model.decode(z_n, neg_edge_index)
        
        pos_labels = torch.ones(pos_score.size(0), device=pos_score.device).float()
        neg_labels = torch.zeros(neg_score.size(0), device=neg_score.device).float()
        
        all_scores.append(pos_score)
        all_scores.append(neg_score)
        all_labels.append(pos_labels)
        all_labels.append(neg_labels)

        all_score = torch.cat([pos_score, neg_score], dim=0)
        all_label = torch.cat([pos_labels, neg_labels], dim=0)

        loss = criterion(all_score, all_label)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        total_loss += loss.item() * (pos_labels.size(0) + neg_labels.size(0))
        total_samples += (pos_labels.size(0) + neg_labels.size(0))

    all_scores = torch.cat(tuple(all_scores), dim=0).detach().cpu().numpy()
    all_labels = torch.cat(tuple(all_labels), dim=0).detach().cpu().numpy()
    predictions = (torch.sigmoid(torch.tensor(all_scores)) > 0.7).int().numpy()

    accuracy = accuracy_score(all_labels, predictions)
    f1 = f1_score(all_labels, predictions)

    return total_loss / total_samples, accuracy, f1

# Testing loop
def test(valid_data):
    model.eval()
    all_scores = []
    all_labels = []
    with torch.no_grad():
        for data in valid_data:
            edge_index = data.edge_index
            data.y[data.y==2] = 1
            z = model(data.x.float(), edge_index)

            out = model.decode(z, edge_index).view(-1).sigmoid()
            
            # Append results to all_scores and all_labels
            all_scores.extend(out.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())  # Assuming data.y is the label tensor

    # Ensure labels are binary
    if len(np.unique(all_labels)) > 2:
        raise ValueError("Expected binary labels but found multi-class labels. Check your data.")
        
    # Calculate metrics
    auc = roc_auc_score(all_labels, all_scores, average="macro")
    predictions = [1 if score > 0.7 else 0 for score in all_scores]
    accuracy = accuracy_score(all_labels, predictions)
    f1 = f1_score(all_labels, predictions)

    return auc, accuracy, f1, predictions, all_labels

# Run training for all epochs
epochs = 200
for epoch in range(epochs):
    train_loss, train_accuracy, train_f1 = train(train_data)
    scheduler.step(train_loss)  # Update learning rate based on training loss
    if epoch % 10 == 0:
        auc, accuracy, f1, predicted, all_labels = test(valid_data)
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Test AUC: {auc:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')


# Final Test
auc, accuracy, f1, predicted, all_labels = test(valid_data)
print(f'Final Test AUC: {auc:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')


/home/jina/anaconda3/envs/jina_gnn/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Loss: 0.7125, Train Accuracy: 0.5274, Train F1 Score: 0.5801, Test AUC: 0.5368, Test Accuracy: 0.5310, Test F1 Score: 0.6063
Epoch 10, Loss: 0.7029, Train Accuracy: 0.5311, Train F1 Score: 0.5651, Test AUC: 0.5515, Test Accuracy: 0.5395, Test F1 Score: 0.5819
Epoch 20, Loss: 0.6932, Train Accuracy: 0.5348, Train F1 Score: 0.5437, Test AUC: 0.5268, Test Accuracy: 0.5250, Test F1 Score: 0.5744
Epoch 30, Loss: 0.6851, Train Accuracy: 0.5466, Train F1 Score: 0.5550, Test AUC: 0.5373, Test Accuracy: 0.5225, Test F1 Score: 0.5377
Epoch 40, Loss: 0.6835, Train Accuracy: 0.5587, Train F1 Score: 0.5718, Test AUC: 0.5335, Test Accuracy: 0.5202, Test F1 Score: 0.5194
Epoch 50, Loss: 0.6767, Train Accuracy: 0.5713, Train F1 Score: 0.5979, Test AUC: 0.5414, Test Accuracy: 0.5257, Test F1 Score: 0.5297
Epoch 60, Loss: 0.6654, Train Accuracy: 0.5725, Train F1 Score: 0.5965, Test AUC: 0.5405, Test Accuracy: 0.5278, Test F1 Score: 0.5304
Epoch 70, Loss: 0.6817, Train Accuracy: 0.5653, Train F1

In [23]:
for a, b in zip(predicted, all_labels):
    print(a, b)

0 1.0
0 1.0
0 1.0
0 1.0
0 1.0
0 0.0
0 1.0
0 1.0
1 1.0
0 0.0
0 1.0
0 0.0
0 1.0
0 1.0
0 1.0
0 1.0
0 0.0
0 1.0
1 0.0
0 1.0
0 0.0
0 0.0
0 1.0
0 0.0
0 1.0
1 1.0
0 1.0
0 0.0
0 1.0
0 1.0
0 0.0
0 1.0
0 0.0
0 0.0
0 1.0
0 0.0
0 0.0
0 1.0
0 0.0
0 0.0
0 0.0
0 1.0
0 1.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 1.0
0 0.0
0 1.0
0 1.0
0 1.0
1 0.0
1 1.0
0 0.0
0 1.0
0 0.0
0 1.0
0 0.0
0 0.0
1 0.0
0 1.0
0 1.0
0 1.0
0 0.0
0 1.0
0 0.0
0 1.0
0 1.0
0 1.0
0 1.0
0 0.0
0 0.0
0 0.0
0 1.0
0 0.0
0 0.0
0 1.0
0 0.0
0 0.0
0 1.0
0 1.0
0 0.0
0 0.0
0 1.0
0 0.0
0 1.0
0 1.0
0 0.0
0 1.0
0 0.0
0 1.0
0 0.0
0 1.0
0 0.0
0 1.0
1 1.0
1 1.0
1 0.0
0 1.0
0 1.0
0 0.0
0 1.0
0 1.0
0 0.0
0 1.0
0 0.0
0 1.0
1 1.0
0 1.0
0 1.0
0 0.0
0 1.0
0 1.0
0 1.0
0 1.0
0 1.0
1 0.0
0 1.0
0 1.0
0 1.0
0 0.0
0 1.0
0 0.0
0 1.0
0 0.0
1 1.0
0 1.0
1 0.0
0 1.0
0 1.0
0 0.0
0 1.0
1 1.0
0 0.0
1 1.0
0 1.0
0 1.0
1 1.0
0 0.0
0 1.0
0 1.0
1 1.0
0 1.0
1 1.0
0 1.0
0 0.0
0 0.0
0 1.0
1 1.0
0 0.0
0 0.0
0 1.0
0 0.0
0 1.0
0 0.0
1 0.0
0 1.0
0 1.0
0 1.0
0 1.0
0 0.0
0 1.0
0 1.

In [24]:
# Convert to numpy arrays for counting values and cast to integer type
predicted_flat = np.array(predicted).astype(int)
all_labels_flat = np.array(all_labels).astype(int)

# Count values
count_predicted = np.bincount(predicted_flat)
count_labels = np.bincount(all_labels_flat)

# Print counts
print("Predicted counts:", count_predicted)
print("Actual labels counts:", count_labels)

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(all_labels_flat, predicted_flat)
precision = precision_score(all_labels_flat, predicted_flat)
recall = recall_score(all_labels_flat, predicted_flat)
f1 = f1_score(all_labels_flat, predicted_flat)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

tn, fp, fn, tp = confusion_matrix(all_labels_flat, predicted_flat).ravel()
print(tn, fp, fn, tp)

Predicted counts: [20780 11415]
Actual labels counts: [14251 17944]
Accuracy: 0.5171
Precision: 0.6050
Recall: 0.3849
F1 Score: 0.4705
9742 4509 11038 6906


In [13]:
from torch_geometric.nn import SAGEConv, LayerNorm
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
import numpy as np 
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, residual=True):
        super(GraphSAGE, self).__init__()
        self.num_layers = num_layers
        self.residual = False  # Residual connection removed for simpler training
        
        # Create convolutional layers dynamically based on num_layers
        self.convs = nn.ModuleList()
        self.lns = nn.ModuleList()
        
        # First layer
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.lns.append(LayerNorm(hidden_channels))
        
        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.lns.append(LayerNorm(hidden_channels))
        
        # Last layer
        self.convs.append(SAGEConv(hidden_channels, out_channels))
        
        # Linear layer for dimension matching if needed
        if residual and in_channels != out_channels:
            self.dim_match = nn.Linear(in_channels, out_channels)
        else:
            self.dim_match = None

    def encode(self, x, edge_index):
        x_res = x  # For potential residual connection
        
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index)
            x = self.lns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        
        # Apply the last convolutional layer without activation for output
        x = self.convs[-1](x, edge_index)
        
        # Residual connection after the last hidden layer if enabled
        if self.residual and self.num_layers > 2:
            if self.dim_match is not None:
                x_res = self.dim_match(x_res)  # Adjust dimension if necessary
            x += x_res

        return x
    
    def decode(self, z, edge_index):
        z_product = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)
        return z_product

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        return z

# Random seed setting
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
        elif isinstance(m, SAGEConv):
            nn.init.xavier_uniform_(m.lin_l.weight)
            if m.lin_r is not None:
                nn.init.xavier_uniform_(m.lin_r.weight)

def seed_everything(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

seed_everything(47)
initialize_weights(model)

# Model, optimizer, loss initialization
in_channels = train_data[0].num_features
hidden_channels = 128  # Reduced hidden channel size for better stability
out_channels = 128  # Increased output channel size for better representation
model = GraphSAGE(in_channels, hidden_channels, out_channels)
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)  # Reduced learning rate for better convergence
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)
criterion = nn.BCEWithLogitsLoss()  # Changed to MarginRankingLoss for better separation  # Using BCEWithLogitsLoss since sigmoid is not applied in decode

def train(train_data):
    model.train()
    total_loss = 0
    total_samples = 0
    all_scores = []
    all_labels = []
    for data in train_data:
        optimizer.zero_grad()
        edge_index = data.edge_index
        data.y[data.y==2] = 1
        z = model(data.x.float(), edge_index)
        
        # Separate positive and negative edges based on labels
        pos_edge_index = data['pos_edge_index'] if 'pos_edge_index' in data else data.edge_index[:, data.y == 1]
        neg_edge_index = data['neg_edge_index'] if 'neg_edge_index' in data else data.edge_index[:, data.y == 0]
        
        # Sample negative edges equal to positive edges
        num_pos = pos_edge_index.size(1)
        perm = torch.randperm(neg_edge_index.size(1))[:num_pos]  # Random permutation for sampling
        neg_edge_index = neg_edge_index[:, perm]
        
        pos_score = model.decode(z, pos_edge_index)
        neg_score = model.decode(z, neg_edge_index)
        
        pos_labels = torch.ones(pos_score.size(0), device=pos_score.device)
        neg_labels = torch.zeros(neg_score.size(0), device=neg_score.device)
        
        target = torch.cat((torch.ones(pos_score.size(0), device=pos_score.device), -torch.ones(neg_score.size(0), device=neg_score.device)), dim=0)
        all_score = torch.cat((pos_score, neg_score), dim=0)
        all_label = torch.cat((pos_labels, neg_labels), dim=0)
        loss = criterion(all_score, all_label)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)  # Gradient clipping
        optimizer.step()
        total_loss += loss.item() * (pos_labels.size(0) + neg_labels.size(0))
        total_samples += (pos_labels.size(0) + neg_labels.size(0))

    return total_loss / total_samples

# Testing loop
def test(valid_data):
    model.eval()
    all_scores = []
    all_labels = []
    with torch.no_grad():
        for data in valid_data:
            edge_index = data.edge_index
            data.y[data.y==2] = 1
            z = model(data.x.float(), edge_index)

            out = model.decode(z, edge_index).view(-1).sigmoid()
            
            # Append results to all_scores and all_labels
            all_scores.extend(out.cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())  # Assuming data.y is the label tensor

    # Ensure labels are binary
    if len(np.unique(all_labels)) > 2:
        raise ValueError("Expected binary labels but found multi-class labels. Check your data.")
        
    # Calculate metrics
    auc = roc_auc_score(all_labels, all_scores, average="macro")
    predictions = [1 if score > 0.5 else 0 for score in all_scores]
    accuracy = accuracy_score(all_labels, predictions)
    f1 = f1_score(all_labels, predictions)

    return auc, accuracy, f1, predictions, all_labels

# Run training for all epochs
epochs = 200
for epoch in range(epochs):
    train_loss = train(train_data)
    scheduler.step()  # Update learning rate based on epoch
    if epoch % 10 == 0:
        auc, accuracy, f1, predicted, all_labels = test(valid_data)
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}, Test AUC: {auc:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')

# Final Test
auc, accuracy, f1, predicted, all_labels = test(valid_data)
print(f'Final Test AUC: {auc:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')



Epoch 0, Loss: 8.9282, Test AUC: 0.5222, Test Accuracy: 0.1660, Test F1 Score: 0.2847
Epoch 10, Loss: 1.9956, Test AUC: 0.6056, Test Accuracy: 0.5149, Test F1 Score: 0.3049
Epoch 20, Loss: 1.3754, Test AUC: 0.5827, Test Accuracy: 0.5319, Test F1 Score: 0.2857
Epoch 30, Loss: 2.2759, Test AUC: 0.6071, Test Accuracy: 0.4681, Test F1 Score: 0.3243
Epoch 40, Loss: 1.0074, Test AUC: 0.6035, Test Accuracy: 0.4553, Test F1 Score: 0.2967
Epoch 50, Loss: 0.9967, Test AUC: 0.5951, Test Accuracy: 0.4298, Test F1 Score: 0.2947
Epoch 60, Loss: 1.3387, Test AUC: 0.6048, Test Accuracy: 0.4723, Test F1 Score: 0.3034
Epoch 70, Loss: 0.9197, Test AUC: 0.6115, Test Accuracy: 0.4851, Test F1 Score: 0.3086
Epoch 80, Loss: 0.9648, Test AUC: 0.6147, Test Accuracy: 0.4766, Test F1 Score: 0.3128
Epoch 90, Loss: 0.9339, Test AUC: 0.6159, Test Accuracy: 0.5064, Test F1 Score: 0.3256
Epoch 100, Loss: 0.7898, Test AUC: 0.6219, Test Accuracy: 0.4851, Test F1 Score: 0.3164
Epoch 110, Loss: 0.8215, Test AUC: 0.6196, 

In [17]:
for a, b in zip(predicted, all_labels):
    print(a, b)

1 0.0
1 0.0
1 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
0 1.0
0 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
0 0.0
0 0.0
0 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
0 0.0
0 0.0
1 0.0
1 0.0
1 0.0
1 1.0
0 0.0
0 0.0
0 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
0 0.0
0 0.0
1 1.0
1 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 1.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
1 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
1 0.0
0 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
0 0.0
0 0.0
1 0.0
0 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
0 0.0
0 0.0
1 1.0
0 0.0
1 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 1.0
1 1.0
1 0.0
0 0.0
0 0.0
1 0.0
0 0.0
1 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
0 0.0
0 0.0
1 0.0
0 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
1 0.0
0 0.0
0 1.0
1 0.0
0 0.0
1 1.0
1 0.0
1 0.0
1 0.0
1 0.0
0 0.0
0 0.0
1 0.0
0 1.0
1 0.0
1 0.0
1 0.0
1 0.0
0 0.0
0 0.0
1 1.0
0 0.0
1 0.0
1 0.0
1 0.0
0 0.0
0 0.

In [19]:
# Convert to numpy arrays for counting values and cast to integer type
predicted_flat = np.array(predicted).astype(int)
all_labels_flat = np.array(all_labels).astype(int)

# Count values
count_predicted = np.bincount(predicted_flat)
count_labels = np.bincount(all_labels_flat)

# Print counts
print("Predicted counts:", count_predicted)
print("Actual labels counts:", count_labels)

# Calculate accuracy and F1 score
accuracy = accuracy_score(all_labels_flat, predicted_flat)
f1 = f1_score(all_labels_flat, predicted_flat)

print(f"Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

Predicted counts: [ 99 136]
Actual labels counts: [196  39]
Accuracy: 0.4936, F1 Score: 0.3200


In [14]:
# Convert to numpy arrays for counting values and cast to integer type
predicted_flat = predicted.astype(int)
all_labels_flat = all_labels.astype(int)

count_predicted = np.bincount(predicted_flat)
count_labels = np.bincount(all_labels_flat)

# Print counts
print("Predicted counts:", count_predicted)
print("Actual labels counts:", count_labels)

# Calculate accuracy and F1 score
accuracy = accuracy_score(all_labels_flat, predicted_flat)
f1 = f1_score(all_labels_flat, predicted_flat)

print(f"Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

AttributeError: 'list' object has no attribute 'astype'

In [38]:
from torch_geometric.nn import SAGEConv, BatchNorm
# from torch_geometric.data import NeighborSampler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
import numpy as np 
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_neighbors):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.bn1 = BatchNorm(hidden_channels)  # Add Batch Normalization
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.num_neighbors = num_neighbors

    def sample_neighbors(self, edge_index, num_nodes, num_neighbors):
        # Custom function to sample neighbors randomly for each node
        sampled_edges = []
        adj_list = [[] for _ in range(num_nodes)]
        for src, dst in edge_index.t().tolist():
            adj_list[src].append(dst)
            adj_list[dst].append(src)
        
        for node in range(num_nodes):
            neighbors = adj_list[node]
            if len(neighbors) > num_neighbors:
                sampled_neighbors = random.sample(neighbors, num_neighbors)
            else:
                sampled_neighbors = neighbors
            for neighbor in sampled_neighbors:
                sampled_edges.append([node, neighbor])
        
        sampled_edge_index = torch.tensor(sampled_edges, dtype=torch.long).t().contiguous()
        return sampled_edge_index

    def encode(self, x, edge_index):
        edge_index = self.sample_neighbors(edge_index, x.size(0), self.num_neighbors)
        x = self.conv1(x, edge_index)
        # x = self.bn1(x)  # Apply Batch Normalization
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)  # Dropout for regularization
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_index):
        z_product = torch.sum(z[edge_index[0]] * z[edge_index[1]], dim=1)
        return z_product / (torch.norm(z[edge_index[0]], dim=1) * torch.norm(z[edge_index[1]], dim=1) + 1e-6)  # 내적 후 코사인 유사도로 변환

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        return z

# Random seed setting
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(47)

# Model, optimizer, loss initialization
in_channels = train_data[0].num_features
hidden_channels = 128
out_channels = 32
num_neighbors = 3  # Number of neighbors to sample during message passing
model = GraphSAGE(in_channels, hidden_channels, out_channels, num_neighbors)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Reduced learning rate for stability
criterion = nn.BCEWithLogitsLoss()  # Using BCEWithLogitsLoss since sigmoid is not applied in decode

# Create data loaders for batching
batch_size = 4
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)

# Training loop
def train(loader):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        edge_index = data.edge_index
        z = model(data.x.float(), edge_index)
        
        # Separate positive and negative edges based on labels
        pos_edge_index = edge_index[:, data.y == 1]
        neg_edge_index = edge_index[:, data.y == 0]
        
        # Balance positive and negative samples
        num_pos = pos_edge_index.size(1)
        neg_edge_index = neg_edge_index[:, :num_pos]  # 샘플링하여 양성 엣지와 균형 맞춤
        
        pos_score = model.decode(z, pos_edge_index)
        neg_score = model.decode(z, neg_edge_index)
        
        pos_labels = torch.ones(pos_score.size(0)).float()
        neg_labels = torch.zeros(neg_score.size(0)).float()
        
        all_scores = torch.cat([pos_score, neg_score], dim=0)
        all_labels = torch.cat([pos_labels, neg_labels], dim=0)

        loss = criterion(all_scores, all_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Testing loop
def test(loader):
    model.eval()
    total_auc = 0
    total_accuracy = 0
    total_f1 = 0
    for data in loader:
        edge_index = data.edge_index
        z = model(data.x.float(), edge_index)
        
        # Separate positive and negative edges based on labels
        pos_edge_index = edge_index[:, data.y == 1]
        neg_edge_index = edge_index[:, data.y == 0]
        
        # # Balance positive and negative samples
        # num_pos = pos_edge_index.size(1)
        # neg_edge_index = neg_edge_index[:, :num_pos]  # 샘플링하여 양성 엣지와 균형 맞춤
        
        with torch.no_grad():
            pos_score = model.decode(z, pos_edge_index)
            neg_score = model.decode(z, neg_edge_index)
        
        pos_labels = torch.ones(pos_score.size(0)).float()
        neg_labels = torch.zeros(neg_score.size(0)).float()
        
        all_scores = torch.cat([pos_score, neg_score], dim=0)
        all_labels = torch.cat([pos_labels, neg_labels], dim=0)
        
        auc = roc_auc_score(all_labels.numpy(), all_scores.numpy())
        predicted = (torch.sigmoid(all_scores) > 0.5).float()  # Apply sigmoid to convert logits to probabilities
        accuracy = accuracy_score(all_labels.numpy(), predicted.numpy())
        f1 = f1_score(all_labels.numpy(), predicted.numpy())

        total_auc += auc
        total_accuracy += accuracy
        total_f1 += f1

    return total_auc / len(loader), total_accuracy / len(loader), total_f1 / len(loader), predicted, all_labels

# Run training for all epochs
epochs = 200
for epoch in range(epochs):
    train_loss = train(train_loader)
    if epoch % 10 == 0:
        auc, accuracy, f1, predicted, all_labels = test(valid_loader)
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}, Test AUC: {auc:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')

# Final Test
auc, accuracy, f1, predicted, all_labels = test(valid_loader)
print(f'Final Test AUC: {auc:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')


Epoch 0, Loss: 0.7043, Test AUC: 0.5133, Test Accuracy: 0.4479, Test F1 Score: 0.5479
Epoch 10, Loss: 0.6992, Test AUC: 0.5247, Test Accuracy: 0.4653, Test F1 Score: 0.5523
Epoch 20, Loss: 0.6991, Test AUC: 0.5028, Test Accuracy: 0.4526, Test F1 Score: 0.5185
Epoch 30, Loss: 0.6992, Test AUC: 0.5231, Test Accuracy: 0.4441, Test F1 Score: 0.5461
Epoch 40, Loss: 0.6972, Test AUC: 0.5147, Test Accuracy: 0.4622, Test F1 Score: 0.5443
Epoch 50, Loss: 0.6973, Test AUC: 0.5224, Test Accuracy: 0.4466, Test F1 Score: 0.5727
Epoch 60, Loss: 0.6978, Test AUC: 0.5209, Test Accuracy: 0.4390, Test F1 Score: 0.5700
Epoch 70, Loss: 0.6968, Test AUC: 0.5054, Test Accuracy: 0.4485, Test F1 Score: 0.5585
Epoch 80, Loss: 0.6994, Test AUC: 0.4948, Test Accuracy: 0.4296, Test F1 Score: 0.5381
Epoch 90, Loss: 0.6976, Test AUC: 0.5207, Test Accuracy: 0.4773, Test F1 Score: 0.5621
Epoch 100, Loss: 0.6976, Test AUC: 0.5052, Test Accuracy: 0.4451, Test F1 Score: 0.5608
Epoch 110, Loss: 0.6986, Test AUC: 0.5151, 

In [39]:
predicted, all_labels

(tensor([1., 1., 1.,  ..., 1., 1., 1.]),
 tensor([1., 1., 1.,  ..., 0., 0., 0.]))

In [146]:
from torch_geometric.nn import SAGEConv, BatchNorm
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_recall_curve
import numpy as np 
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.loader import DataLoader  # Updated DataLoader import to avoid deprecation warning

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, dropout=0.3, edge_feat_dim=0):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.num_layers = num_layers
        self.dropout = dropout

        # Adjust input dimension if edge features are used
        input_channels = in_channels
        if edge_feat_dim > 0:
            self.edge_linear = nn.Linear(edge_feat_dim, in_channels).float().to(device)
            input_channels += in_channels  # Adding edge features after transformation

        # Input layer
        self.layers.append(SAGEConv(input_channels, hidden_channels))
        self.batch_norms.append(BatchNorm(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(SAGEConv(hidden_channels, hidden_channels))
            self.batch_norms.append(BatchNorm(hidden_channels))

        # Output layer
        self.layers.append(SAGEConv(hidden_channels, out_channels))

    def encode(self, x, edge_index, edge_attr=None):
        for i in range(self.num_layers - 1):
            residual = x  # Residual Connection
            if edge_attr is not None:
                # Transform edge features and add to corresponding nodes
                transformed_edge_attr = self.edge_linear(edge_attr.float().to(device))
                edge_node_features = torch.zeros((x.size(0), transformed_edge_attr.size(1)), device=x.device)

                # Scatter edge features to nodes using index_add for better handling of sizes
                v, u = edge_index
                edge_node_features.index_add_(0, v, transformed_edge_attr)
                edge_node_features.index_add_(0, u, transformed_edge_attr)

                # Concatenate original node features with edge information
                x = torch.cat([x, edge_node_features], dim=-1)

            # Adjust the input dimension of the SAGEConv layer to match the new input size dynamically
            if x.size(1) != self.layers[i].in_channels:
                self.layers[i] = SAGEConv(x.size(1), self.layers[i].out_channels).to(device)

            x = self.layers[i](x, edge_index)
            x = self.batch_norms[i](x)
            if residual.size() == x.size():  # Check if residual connection is possible
                x = F.relu(x) + residual  # Adding residual connection
            else:
                x = F.relu(x)  # Skip residual if dimensions do not match
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.layers[-1](x, edge_index)
        return x

    def decode(self, z, edge_index, neighbor_embeddings=None):
        # Using normalized dot product for prediction to avoid extreme values
        z = F.normalize(z, p=2, dim=1)
        if neighbor_embeddings is not None:
            neighbor_embeddings = F.normalize(neighbor_embeddings, p=2, dim=1)
            return torch.sum(z[edge_index[0]] * z[edge_index[1]] * neighbor_embeddings, dim=1)
        return torch.sum(z[edge_index[0]] * z[edge_index[1]], dim=1)

    def forward(self, x, edge_index, edge_attr=None):
        z = self.encode(x, edge_index, edge_attr)
        return z

# Random seed setting
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(47)

# Model, optimizer, loss initialization
in_channels = train_data[0].num_features
hidden_channels = 512  # Adjusted hidden channel size to match common practice (powers of 2)
out_channels = 128
edge_feat_dim = train_data[0].edge_attr.size(1) if 'edge_attr' in train_data[0] else 0
model = GraphSAGE(in_channels, hidden_channels, out_channels, num_layers=4, dropout=0.3, edge_feat_dim=edge_feat_dim).to(device)  # Increased dropout to 0.3
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)  # Adjusted weight decay
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-5)  # Cosine Annealing Scheduler for gradual learning rate changes
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.0], dtype=torch.float).to(device))  # Added class weighting with correct dtype

# DataLoader initialization
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)  # Mini-batch and shuffle
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)

# Training loop
def train(train_loader):
    model.train()
    total_loss = 0
    total_accuracy = 0
    total_f1 = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        edge_index = data.edge_index.to(device)
        edge_attr = data.edge_attr.to(device) if 'edge_attr' in data else None
        z = model(data.x.float().to(device), edge_index, edge_attr)
        
        # Separate positive and negative edges based on labels
        pos_edge_index = edge_index[:, data.y == 1]
        neg_edge_index = edge_index[:, data.y == 0]
        
        # Ensure equal number of negative samples as positive samples
        num_pos = pos_edge_index.size(1)
        if neg_edge_index.size(1) > num_pos:  # Match negative samples to positive samples
            perm = torch.randperm(neg_edge_index.size(1))[:num_pos]  # Random permutation for sampling
            neg_edge_index = neg_edge_index[:, perm]
        
        pos_score = model.decode(z, pos_edge_index)
        neg_score = model.decode(z, neg_edge_index)
        
        pos_labels = torch.ones(pos_score.size(0)).float().to(device)
        neg_labels = torch.zeros(neg_score.size(0)).float().to(device)
        
        all_scores = torch.cat([pos_score, neg_score], dim=0)
        all_labels = torch.cat([pos_labels, neg_labels], dim=0)

        loss = criterion(all_scores, all_labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        total_loss += loss.item()

        # Calculate training accuracy and F1 score
        with torch.no_grad():
            predicted = (torch.sigmoid(all_scores) > 0.5).float()  # Apply sigmoid to convert logits to probabilities
            accuracy = accuracy_score(all_labels.cpu().numpy(), predicted.cpu().numpy())
            f1 = f1_score(all_labels.cpu().numpy(), predicted.cpu().numpy())
            total_accuracy += accuracy
            total_f1 += f1

    return total_loss / len(train_loader), total_accuracy / len(train_loader), total_f1 / len(train_loader)

# Testing loop
def test(valid_loader):
    model.eval()
    total_auc = 0
    total_accuracy = 0
    total_f1 = 0
    all_predictions = []
    all_true_labels = []
    for data in valid_loader:
        data = data.to(device)
        edge_index = data.edge_index.to(device)
        edge_attr = data.edge_attr.to(device) if 'edge_attr' in data else None
        z = model(data.x.float().to(device), edge_index, edge_attr)
        
        # Separate positive and negative edges based on labels
        pos_edge_index = edge_index[:, data.y == 1]
        neg_edge_index = edge_index[:, data.y == 0]
        
        # Ensure equal number of negative samples as positive samples
        # num_pos = pos_edge_index.size(1)
        # if neg_edge_index.size(1) > num_pos:  # Randomly select negative samples to match positive samples
        #     perm = torch.randperm(neg_edge_index.size(1))[:num_pos]  # Random permutation for sampling
        #     neg_edge_index = neg_edge_index[:, perm]
        
        with torch.no_grad():
            pos_score = model.decode(z, pos_edge_index)
            neg_score = model.decode(z, neg_edge_index)
        
        pos_labels = torch.ones(pos_score.size(0)).float().to(device)
        neg_labels = torch.zeros(neg_score.size(0)).float().to(device)
        all_scores = torch.cat([pos_score, neg_score], dim=0)
        all_labels = torch.cat([pos_labels, neg_labels], dim=0)
        
        # Calculate AUC, accuracy, and F1 score
        auc = roc_auc_score(all_labels.cpu().numpy(), torch.sigmoid(all_scores).cpu().numpy())
        accuracy = accuracy_score(all_labels.cpu().numpy(), (torch.sigmoid(all_scores) > 0.5).cpu().numpy())
        f1 = f1_score(all_labels.cpu().numpy(), (torch.sigmoid(all_scores) > 0.5).cpu().numpy())
        total_auc += auc
        total_accuracy += accuracy
        total_f1 += f1

    return total_auc / len(valid_loader), total_accuracy / len(valid_loader), total_f1 / len(valid_loader)

# Training process
epochs = 200
for epoch in range(epochs):
    train_loss, train_accuracy, train_f1 = train(train_loader)
    valid_auc, valid_accuracy, valid_f1 = test(valid_loader)
    scheduler.step()  # Update learning rate based on scheduler

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Test AUC: {valid_auc:.4f}, Test Accuracy: {valid_accuracy:.4f}, Test F1 Score: {valid_f1:.4f}")

final_auc, final_accuracy, final_f1 = test(valid_loader)
print(f"Final Test AUC: {final_auc:.4f}, Test Accuracy: {final_accuracy:.4f}, Test F1 Score: {final_f1:.4f}")


Epoch 0, Loss: 0.7287, Train Accuracy: 0.5005, Train F1 Score: 0.6663, Test AUC: 0.4882, Test Accuracy: 0.3886, Test F1 Score: 0.5597
Epoch 10, Loss: 0.6933, Train Accuracy: 0.5144, Train F1 Score: 0.6406, Test AUC: 0.4840, Test Accuracy: 0.3891, Test F1 Score: 0.5587
Epoch 20, Loss: 0.6957, Train Accuracy: 0.5057, Train F1 Score: 0.6274, Test AUC: 0.4838, Test Accuracy: 0.3982, Test F1 Score: 0.5537
Epoch 30, Loss: 0.6900, Train Accuracy: 0.5216, Train F1 Score: 0.6223, Test AUC: 0.4875, Test Accuracy: 0.3990, Test F1 Score: 0.5529
Epoch 40, Loss: 0.6926, Train Accuracy: 0.5171, Train F1 Score: 0.6184, Test AUC: 0.4858, Test Accuracy: 0.3906, Test F1 Score: 0.5559
Epoch 50, Loss: 0.6921, Train Accuracy: 0.5150, Train F1 Score: 0.6077, Test AUC: 0.4827, Test Accuracy: 0.4002, Test F1 Score: 0.5539
Epoch 60, Loss: 0.6910, Train Accuracy: 0.5168, Train F1 Score: 0.6038, Test AUC: 0.4818, Test Accuracy: 0.4001, Test F1 Score: 0.5535
Epoch 70, Loss: 0.6906, Train Accuracy: 0.5146, Train F1

In [147]:
for a, b in zip(predicted, all_labels):
    print(a, b)

tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.)

In [ ]:
from torch_geometric.nn import SAGEConv, BatchNorm
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_recall_curve
import numpy as np 
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, dropout=0.3):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.num_layers = num_layers
        self.dropout = dropout

        # Input layer
        self.layers.append(SAGEConv(in_channels, hidden_channels))
        self.batch_norms.append(BatchNorm(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(SAGEConv(hidden_channels, hidden_channels))
            self.batch_norms.append(BatchNorm(hidden_channels))

        # Output layer
        self.layers.append(SAGEConv(hidden_channels, out_channels))

    def encode(self, x, edge_index):
        for i in range(self.num_layers - 1):
            residual = x  # Residual Connection
            x = self.layers[i](x, edge_index)
            x = self.batch_norms[i](x)
            if residual.size() == x.size():  # Check if residual connection is possible
                x = F.relu(x) + residual  # Adding residual connection
            else:
                x = F.relu(x)  # Skip residual if dimensions do not match
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.layers[-1](x, edge_index)
        return x

    def decode(self, z, edge_index):
        # Using normalized dot product for prediction to avoid extreme values
        z = F.normalize(z, p=2, dim=1)
        return torch.sum(z[edge_index[0]] * z[edge_index[1]], dim=1)

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        return z

# Random seed setting
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(47)

# Model, optimizer, loss initialization
in_channels = train_data[0].num_features
hidden_channels = 512  # Increased hidden channel size
out_channels = 128
model = GraphSAGE(in_channels, hidden_channels, out_channels, num_layers=4, dropout=0.3).to(device)  # Increased dropout to 0.3
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)  # Adjusted weight decay
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-5)  # Cosine Annealing Scheduler for gradual learning rate changes
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.0]).to(device))  # Added class weighting

# DataLoader initialization
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)  # Mini-batch and shuffle
valid_loader = DataLoader(valid_data, batch_size=4, shuffle=False)

# Training loop
def train(train_loader):
    model.train()
    total_loss = 0
    total_accuracy = 0
    total_f1 = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        edge_index = data.edge_index
        z = model(data.x.float(), edge_index)
        
        # Separate positive and negative edges based on labels
        pos_edge_index = edge_index[:, data.y == 1]
        neg_edge_index = edge_index[:, data.y == 0]
        
        # Ensure equal number of negative samples as positive samples
        num_pos = pos_edge_index.size(1)
        if neg_edge_index.size(1) > num_pos:  # Match negative samples to positive samples
            neg_edge_index = neg_edge_index[:, :num_pos] 
        
        pos_score = model.decode(z, pos_edge_index)
        neg_score = model.decode(z, neg_edge_index)
        
        pos_labels = torch.ones(pos_score.size(0)).float().to(device)
        neg_labels = torch.zeros(neg_score.size(0)).float().to(device)
        
        all_scores = torch.cat([pos_score, neg_score], dim=0)
        all_labels = torch.cat([pos_labels, neg_labels], dim=0)

        loss = criterion(all_scores, all_labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        total_loss += loss.item()

        # Calculate training accuracy and F1 score
        with torch.no_grad():
            predicted = (torch.sigmoid(all_scores) > 0.5).float()  # Apply sigmoid to convert logits to probabilities
            accuracy = accuracy_score(all_labels.cpu().numpy(), predicted.cpu().numpy())
            f1 = f1_score(all_labels.cpu().numpy(), predicted.cpu().numpy())
            total_accuracy += accuracy
            total_f1 += f1

    return total_loss / len(train_loader), total_accuracy / len(train_loader), total_f1 / len(train_loader)

# Testing loop
def test(valid_loader):
    model.eval()
    total_auc = 0
    total_accuracy = 0
    total_f1 = 0
    all_predictions = []
    all_true_labels = []
    for data in valid_loader:
        data = data.to(device)
        edge_index = data.edge_index
        z = model(data.x.float(), edge_index)
        
        # Separate positive and negative edges based on labels
        pos_edge_index = edge_index[:, data.y == 1]
        neg_edge_index = edge_index[:, data.y == 0]
        
        # Ensure equal number of negative samples as positive samples
    #     num_pos = pos_edge_index.size(1)
    # if neg_edge_index.size(1) > num_pos:  # Randomly select negative samples to match positive samples
    #     perm = torch.randperm(neg_edge_index.size(1))[:num_pos]  # Random permutation for sampling
    #     neg_edge_index = neg_edge_index[:, perm]
        
        with torch.no_grad():
            pos_score = model.decode(z, pos_edge_index)
            neg_score = model.decode(z, neg_edge_index)
        
        pos_labels = torch.ones(pos_score.size(0)).float().to(device)
        neg_labels = torch.zeros(neg_score.size(0)).float().to(device)
        
        all_scores = torch.cat([pos_score, neg_score], dim=0)
        all_labels = torch.cat([pos_labels, neg_labels], dim=0)
        
        auc = roc_auc_score(all_labels.cpu().numpy(), all_scores.cpu().numpy())
        precision, recall, thresholds = precision_recall_curve(all_labels.cpu().numpy(), torch.sigmoid(all_scores).cpu().numpy())
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-6)
        best_threshold = thresholds[np.argmax(f1_scores)]  # Find the best threshold for F1 score

        predicted = (torch.sigmoid(all_scores) > best_threshold).float()  # Apply optimized threshold
        accuracy = accuracy_score(all_labels.cpu().numpy(), predicted.cpu().numpy())
        f1 = f1_score(all_labels.cpu().numpy(), predicted.cpu().numpy())

        total_auc += auc
        total_accuracy += accuracy
        total_f1 += f1

        # Store all predictions and true labels
        all_predictions.append(predicted)
        all_true_labels.append(all_labels)

    # Concatenate all predictions and true labels
    all_predictions = torch.cat(all_predictions, dim=0)
    all_true_labels = torch.cat(all_true_labels, dim=0)

    return total_auc / len(valid_loader), total_accuracy / len(valid_loader), total_f1 / len(valid_loader), all_predictions, all_true_labels

# Run training for all epochs
epochs = 200
for epoch in range(epochs):
    train_loss, train_accuracy, train_f1 = train(train_loader)
    scheduler.step()  # Update learning rate based on scheduler
    if epoch % 10 == 0:
        auc, accuracy, f1, predicted, all_labels = test(valid_loader)
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Test AUC: {auc:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')

# Final Test
auc, accuracy, f1, predicted, all_labels = test(valid_loader)
print(f'Final Test AUC: {auc:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')


/home/jina/anaconda3/envs/jina_gnn/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 0, Loss: 0.7063, Train Accuracy: 0.5127, Train F1 Score: 0.6688, Test AUC: 0.0975, Test Accuracy: 0.1002, Test F1 Score: 0.1334
Epoch 10, Loss: 0.6768, Train Accuracy: 0.5840, Train F1 Score: 0.6697, Test AUC: 0.0969, Test Accuracy: 0.1002, Test F1 Score: 0.1333
Epoch 20, Loss: 0.6611, Train Accuracy: 0.5998, Train F1 Score: 0.6659, Test AUC: 0.0965, Test Accuracy: 0.1000, Test F1 Score: 0.1333
Epoch 30, Loss: 0.6664, Train Accuracy: 0.5953, Train F1 Score: 0.6597, Test AUC: 0.0973, Test Accuracy: 0.1000, Test F1 Score: 0.1333
Epoch 40, Loss: 0.6579, Train Accuracy: 0.6054, Train F1 Score: 0.6631, Test AUC: 0.0959, Test Accuracy: 0.1000, Test F1 Score: 0.1333
Epoch 50, Loss: 0.6590, Train Accuracy: 0.5977, Train F1 Score: 0.6528, Test AUC: 0.0982, Test Accuracy: 0.1002, Test F1 Score: 0.1333
Epoch 60, Loss: 0.6659, Train Accuracy: 0.5942, Train F1 Score: 0.6533, Test AUC: 0.0969, Test Accuracy: 0.1000, Test F1 Score: 0.1333
Epoch 70, Loss: 0.6704, Train Accuracy: 0.5885, Train F1

In [144]:
for a, b in zip(predicted, all_labels):
    print(a, b)

tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(0.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
tensor(1.)

KeyboardInterrupt: 

In [ ]:
from torch_geometric.nn import SAGEConv, BatchNorm
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_recall_curve
import numpy as np 
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.loader import DataLoader  # Updated DataLoader import to avoid deprecation warning

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, dropout=0.3, edge_feat_dim=0):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.num_layers = num_layers
        self.dropout = dropout

        # Adjust input dimension if edge features are used
        input_channels = in_channels
        if edge_feat_dim > 0:
            self.edge_linear = nn.Linear(edge_feat_dim, in_channels).float().to(device)
            input_channels += in_channels  # Adding edge features after transformation

        # Input layer
        self.layers.append(SAGEConv(input_channels, hidden_channels))
        self.batch_norms.append(BatchNorm(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(SAGEConv(hidden_channels, hidden_channels))
            self.batch_norms.append(BatchNorm(hidden_channels))

        # Output layer
        self.layers.append(SAGEConv(hidden_channels, out_channels))

    def encode(self, x, edge_index, edge_attr=None):
        for i in range(self.num_layers - 1):
            residual = x  # Residual Connection
            if edge_attr is not None:
                # Transform edge features and add to corresponding nodes
                transformed_edge_attr = self.edge_linear(edge_attr.float().to(device))
                edge_node_features = torch.zeros((x.size(0), transformed_edge_attr.size(1)), device=x.device)

                # Scatter edge features to nodes using index_add for better handling of sizes
                v, u = edge_index
                edge_node_features.index_add_(0, v, transformed_edge_attr)
                edge_node_features.index_add_(0, u, transformed_edge_attr)

                # Concatenate original node features with edge information
                x = torch.cat([x, edge_node_features], dim=-1)

            # Adjust the input dimension of the SAGEConv layer to match the new input size dynamically
            if x.size(1) != self.layers[i].in_channels:
                self.layers[i] = SAGEConv(x.size(1), self.layers[i].out_channels).to(device)

            x = self.layers[i](x, edge_index)
            x = self.batch_norms[i](x)
            if residual.size() == x.size():  # Check if residual connection is possible
                x = F.relu(x) + residual  # Adding residual connection
            else:
                x = F.relu(x)  # Skip residual if dimensions do not match
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.layers[-1](x, edge_index)
        return x

    def decode(self, z, edge_index, neighbor_embeddings=None):
        # Using normalized dot product for prediction to avoid extreme values
        z = F.normalize(z, p=2, dim=1)
        if neighbor_embeddings is not None:
            neighbor_embeddings = F.normalize(neighbor_embeddings, p=2, dim=1)
            return torch.sum(z[edge_index[0]] * z[edge_index[1]] * neighbor_embeddings, dim=1)
        return torch.sum(z[edge_index[0]] * z[edge_index[1]], dim=1)

    def forward(self, x, edge_index, edge_attr=None):
        z = self.encode(x, edge_index, edge_attr)
        return z

# Random seed setting
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(47)

# Model, optimizer, loss initialization
in_channels = train_data[0].num_features
hidden_channels = 512  # Adjusted hidden channel size to match common practice (powers of 2)
out_channels = 128
edge_feat_dim = train_data[0].edge_attr.size(1) if 'edge_attr' in train_data[0] else 0
model = GraphSAGE(in_channels, hidden_channels, out_channels, num_layers=4, dropout=0.3, edge_feat_dim=edge_feat_dim).to(device)  # Increased dropout to 0.3
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)  # Adjusted weight decay
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-5)  # Cosine Annealing Scheduler for gradual learning rate changes
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.0], dtype=torch.float).to(device))  # Added class weighting with correct dtype

# DataLoader initialization
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)  # Mini-batch and shuffle
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)

# Training loop
def train(train_loader):
    model.train()
    total_loss = 0
    total_accuracy = 0
    total_f1 = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        edge_index = data.edge_index.to(device)
        edge_attr = data.edge_attr.to(device) if 'edge_attr' in data else None
        z = model(data.x.float().to(device), edge_index, edge_attr)
        
        # Separate positive and negative edges based on labels
        pos_edge_index = edge_index[:, data.y == 1]
        neg_edge_index = edge_index[:, data.y == 0]
        
        # Ensure equal number of negative samples as positive samples
        num_pos = pos_edge_index.size(1)
        if neg_edge_index.size(1) > num_pos:  # Match negative samples to positive samples
            perm = torch.randperm(neg_edge_index.size(1))[:num_pos]  # Random permutation for sampling
            neg_edge_index = neg_edge_index[:, perm]
        
        pos_score = model.decode(z, pos_edge_index)
        neg_score = model.decode(z, neg_edge_index)
        
        pos_labels = torch.ones(pos_score.size(0)).float().to(device)
        neg_labels = torch.zeros(neg_score.size(0)).float().to(device)
        
        all_scores = torch.cat([pos_score, neg_score], dim=0)
        all_labels = torch.cat([pos_labels, neg_labels], dim=0)

        loss = criterion(all_scores, all_labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        total_loss += loss.item()

        # Calculate training accuracy and F1 score
        with torch.no_grad():
            predicted = (torch.sigmoid(all_scores) > 0.5).float()  # Apply sigmoid to convert logits to probabilities
            accuracy = accuracy_score(all_labels.cpu().numpy(), predicted.cpu().numpy())
            f1 = f1_score(all_labels.cpu().numpy(), predicted.cpu().numpy())
            total_accuracy += accuracy
            total_f1 += f1

    return total_loss / len(train_loader), total_accuracy / len(train_loader), total_f1 / len(train_loader)

# Testing loop
def test(valid_loader):
    model.eval()
    total_auc = 0
    total_accuracy = 0
    total_f1 = 0
    all_predictions = []
    all_true_labels = []
    for data in valid_loader:
        data = data.to(device)
        edge_index = data.edge_index.to(device)
        edge_attr = data.edge_attr.to(device) if 'edge_attr' in data else None
        z = model(data.x.float().to(device), edge_index, edge_attr)
        
        # Separate positive and negative edges based on labels
        pos_edge_index = edge_index[:, data.y == 1]
        neg_edge_index = edge_index[:, data.y == 0]
        
        # Ensure equal number of negative samples as positive samples
        # num_pos = pos_edge_index.size(1)
        # if neg_edge_index.size(1) > num_pos:  # Randomly select negative samples to match positive samples
        #     perm = torch.randperm(neg_edge_index.size(1))[:num_pos]  # Random permutation for sampling
        #     neg_edge_index = neg_edge_index[:, perm]
        
        with torch.no_grad():
            pos_score = model.decode(z, pos_edge_index)
            neg_score = model.decode(z, neg_edge_index)
        
        pos_labels = torch.ones(pos_score.size(0)).float().to(device)
        neg_labels = torch.zeros(neg_score.size(0)).float().to(device)
        all_scores = torch.cat([pos_score, neg_score], dim=0)
        all_labels = torch.cat([pos_labels, neg_labels], dim=0)
        all_true_labels.append(all_labels)
        all_predictions.append(torch.sigmoid(all_scores) > 0.5)
        
        # Calculate AUC, accuracy, and F1 score
        auc = roc_auc_score(all_labels.cpu().numpy(), torch.sigmoid(all_scores).cpu().numpy())
        accuracy = accuracy_score(all_labels.cpu().numpy(), (torch.sigmoid(all_scores) > 0.5).cpu().numpy())
        f1 = f1_score(all_labels.cpu().numpy(), (torch.sigmoid(all_scores) > 0.5).cpu().numpy())
        total_auc += auc
        total_accuracy += accuracy
        total_f1 += f1

    return total_auc / len(valid_loader), total_accuracy / len(valid_loader), total_f1 / len(valid_loader), all_predictions, all_true_labels

# Training process
epochs = 200
for epoch in range(epochs):
    train_loss, train_accuracy, train_f1 = train(train_loader)
    valid_auc, valid_accuracy, valid_f1, predicted, all_labels = test(valid_loader)
    scheduler.step()  # Update learning rate based on scheduler

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Test AUC: {valid_auc:.4f}, Test Accuracy: {valid_accuracy:.4f}, Test F1 Score: {valid_f1:.4f}")

final_auc, final_accuracy, final_f1 = test(valid_loader)
print(f"Final Test AUC: {final_auc:.4f}, Test Accuracy: {final_accuracy:.4f}, Test F1 Score: {final_f1:.4f}")

Epoch 0, Loss: 0.7287, Train Accuracy: 0.5005, Train F1 Score: 0.6663, Test AUC: 0.4804, Test Accuracy: 0.5000, Test F1 Score: 0.6667
Epoch 10, Loss: 0.6955, Train Accuracy: 0.5135, Train F1 Score: 0.6408, Test AUC: 0.4839, Test Accuracy: 0.4983, Test F1 Score: 0.6629
Epoch 20, Loss: 0.6919, Train Accuracy: 0.5160, Train F1 Score: 0.6228, Test AUC: 0.4818, Test Accuracy: 0.5004, Test F1 Score: 0.6576
Epoch 30, Loss: 0.6914, Train Accuracy: 0.5138, Train F1 Score: 0.6239, Test AUC: 0.4916, Test Accuracy: 0.4987, Test F1 Score: 0.6612
Epoch 40, Loss: 0.6911, Train Accuracy: 0.5182, Train F1 Score: 0.6075, Test AUC: 0.4830, Test Accuracy: 0.4984, Test F1 Score: 0.6605
Epoch 50, Loss: 0.6915, Train Accuracy: 0.5178, Train F1 Score: 0.6145, Test AUC: 0.4846, Test Accuracy: 0.5023, Test F1 Score: 0.6579
Epoch 60, Loss: 0.6908, Train Accuracy: 0.5155, Train F1 Score: 0.6047, Test AUC: 0.4798, Test Accuracy: 0.4967, Test F1 Score: 0.6599
Epoch 70, Loss: 0.6932, Train Accuracy: 0.5074, Train F1

In [140]:
for a, b in zip(predicted, all_labels):
    print(a, b)

tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='cuda:0')
tensor(1., device='cuda:0') tensor(1., device='c

In [46]:
train_data[0].edge_attr.shape

torch.Size([190, 100])

In [47]:
train_data[0].edge_index.shape

torch.Size([2, 190])

In [48]:
train_data[0].num_nodes

20

In [56]:
train_data[0].edge_index[0][2], train_data[0].edge_index[1][2]

(tensor(0), tensor(3))

In [59]:
train_data[0].edge_attr[2].shape

torch.Size([100])

In [63]:
ss = train_data[0]

In [66]:
ss.edge_attr[ss.edge_index[0]]

tensor([[1.1748e+00, 1.1748e+00, 4.3293e-03,  ..., 4.6998e-02, 3.7581e-01,
         3.7581e-01],
        [1.1748e+00, 1.1748e+00, 4.3293e-03,  ..., 4.6998e-02, 3.7581e-01,
         3.7581e-01],
        [1.1748e+00, 1.1748e+00, 4.3293e-03,  ..., 4.6998e-02, 3.7581e-01,
         3.7581e-01],
        ...,
        [4.5446e+00, 4.5446e+00, 1.1615e-01,  ..., 3.2423e-02, 2.7071e-01,
         2.7071e-01],
        [4.5446e+00, 4.5446e+00, 1.1615e-01,  ..., 3.2423e-02, 2.7071e-01,
         2.7071e-01],
        [1.1125e+00, 1.1125e+00, 8.9737e-03,  ..., 3.0051e-02, 3.5498e-01,
         3.5498e-01]], dtype=torch.float64)